In [13]:
%pip install nbformat

%run PSO.ipynb #da qui possiamousare la funzione PSO

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

train_sample = X_train[:1000]
train_sample_labels = y_train[:1000]

test_sample = X_test[:200]
test_sample_labels = y_test[:200]

X_valid = X_train[1000:1250]
y_valid = y_train[1000:1250]

In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((train_sample,train_sample_labels))
valid_ds=tf.data.Dataset.from_tensor_slices((X_valid,y_valid))
test_ds=tf.data.Dataset.from_tensor_slices((test_sample,test_sample_labels))

In [ ]:
def process_image(image,label):
    image=tf.image.per_image_standardization(image)
    image=tf.image.resize(image,(64,64))
    
    return image,label

In [ ]:
model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(64,64,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation='softmax')  
    ])

In [ ]:
# mi serve una funzione che prenda in input solo i parametri della pso e restituisca l'accuracy

def CNN(batch_size, optimizer, learning_rate, train_ds = train_ds, valid_ds = valid_ds, model = model):

    train_ds_size=tf.data.experimental.cardinality(train_ds).numpy()
    valid_ds_size=tf.data.experimental.cardinality(valid_ds).numpy()

    train_ds = (train_ds.map(process_image).shuffle(buffer_size = train_ds_size).batch(batch_size=int(batch_size), drop_reminder = True))
    valid_ds = (valid_ds.map(process_image).shuffle(buffer_size = valid_ds_size).batch(batch_size=int(batch_size), drop_reminder = True))

    if optimizer < 1:
        #use SGD
        opt = tf.keras.optimizers.experimental.SGD(lr = learning_rate)
        
    if optimizer<=1 and optimizer < 2:
        #use SGD with Momentum
        opt = tf.keras.optimizers.experimental.SGD(momentum = 0.9, lr = learning_rate)

    if optimizer >= 2 and optimizer < 3:
        #use ADAM
        opt = tf.keras.optimizers.Adam(lr = learning_rate)

    if optimizer >= 3 and optimizer < 4:
        #use RMSPROP
        opt = tf.keras.optimizers.experimental.RMSprop(lr = learning_rate)

    model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= opt,
    metrics=['accuracy'])

    print(model.summary())

    history=model.fit(
    train_ds,
    epochs=5,
    validation_data=valid_ds,
    validation_freq=1)

    last_accuracy = history.history['val_accuracy'][-1]

    return last_accuracy


Best position found:  [-0.0200253   0.01446194 -0.0031102 ] with an evaluation of:  0.0006198339423027385


In [14]:
# Nella funzione CNN la prima componente è il batch_size, che varia tra 0 e 128.
# La seconda componente è l'optimizer che varia tra 0 e 4. (tra 0 e 1 usa sgd, tra 1 e 2 usa sgd with momenutm, tra 2 e 3 usa adam e tra 3 e 4 usa rmsprop).
# La terza componente il learning rate che varia tra 0.001 e 0.01

PSO(swarm_size = 10,
    dim = 3,
    evaluation_funct = CNN,
    lower_bound=[0,0,0.0001],
    upper_bound = [128,4,0.01],
    v_max = [64,2,0.001],
    problem = 'maximum',
    max_iteration = 200)

Best position found:  [ 1.72270273e-02  4.61055967e-05 -1.70497164e-02] with an evaluation of:  0.0005874654252639145
